In [23]:
import numpy as np
import pandas as pd
import pymysql
import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pickle
import matplotlib.pyplot as plt
from constants import * 

In [24]:
# make database connection
try:
    # Connect to the database
    connection = pymysql.connect(host=DB,
                                user=DB_USER,
                                password=DB_PW,
                                database=CITY)
except pymysql.Error as e:
    print(e)

In [25]:
# get availability
get_availability = "SELECT * FROM availability"
df_availability = pd.read_sql_query(get_availability, connection)

# get weather
get_weather = "SELECT * FROM weather"
df_weather = pd.read_sql_query(get_weather, connection)

connection.close()

/tmp/ipykernel_23042/1168624268.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_availability = pd.read_sql_query(get_availability, connection)
/tmp/ipykernel_23042/1168624268.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_weather = pd.read_sql_query(get_weather, connection)


In [26]:
df_availability.head()

,number,available_bikes,available_bike_stands,last_update
0,1,15,16,1710576480
1,1,15,16,1710577085
2,1,16,15,1710577691
3,1,16,15,1710578296
4,1,16,15,1710578901


In [27]:
df_weather.head()

,last_update,rain,temp,hum
0,1712652300,0.03,7.0,81.0
1,1712653200,0.11,7.0,81.0
2,1712654100,0.11,8.0,76.0
3,1712655000,0.11,8.0,76.0
4,1712655900,0.11,8.0,76.0


In [28]:
# merge availability with weather data
df = pd.merge_asof(df_availability.sort_values('last_update'), 
                          df_weather.sort_values('last_update'), 
                          on='last_update')

df.head()

,number,available_bikes,available_bike_stands,last_update,rain,temp,hum
0,10,11,5,1710576107,NaN,NaN,NaN
1,95,38,2,1710576111,NaN,NaN,NaN
2,60,14,16,1710576117,NaN,NaN,NaN
3,20,1,29,1710576118,NaN,NaN,NaN
4,105,3,33,1710576118,NaN,NaN,NaN


Unfortunately, the scraper that gathered the weather data was failing for the first few weeks and we weren't aware, so the data we have available isn't as comprehensive as we would like

In [29]:
df = df.dropna()
df.shape

(150221, 7)

The day of the week and the hour of the day are likely useful predictors for bike availability, and as such we want to impute these from the data collected

In [30]:
def get_day_and_hour(timestamp):
    datetime_obj = pd.to_datetime(timestamp, unit='s')
    return datetime_obj.hour, datetime_obj.dayofweek

In [31]:
df['day'], df['hour'] = zip(*df['last_update'].apply(get_day_and_hour))

Additionally, instead of prpedicting both `available_bikes` and `available_parking`, it makes instead to predict for just `availability`, which is effectively a ratio of the available bikes to the overall capacity of the station

This simplifies the analytics pipeline and will make the predictions more intutive: high availability is desirable in a departure station and low availability (high parking) is desirable in an arrival station

In [32]:
df['availability'] = df['available_bikes'] / (df['available_bikes'] + df['available_bike_stands'])

In [33]:
df.head()

,number,available_bikes,available_bike_stands,last_update,rain,temp,hum,day,hour,availability
460665,97,8,32,1712652301,0.03,7.0,81.0,8,1,0.200000
460666,53,19,21,1712652306,0.03,7.0,81.0,8,1,0.475000
460667,58,38,2,1712652313,0.03,7.0,81.0,8,1,0.950000
460668,24,10,10,1712652319,0.03,7.0,81.0,8,1,0.500000
460669,44,2,28,1712652325,0.03,7.0,81.0,8,1,0.066667


We will be making predictions for each station based on `rain`, `temp`, `hum`, `day` and `hour`: `day` is a categorical variable and we will treat the others as continuous

The first stage of our pipeline will be declaring `day` as a categorical variable

It is also best practise to standardise the scale of the continuous features

And finally, to add some non-linearity, we will impute some polynomial features for the continuous features

In [34]:
categorical = ['day']
continuous = ['rain', 'temp', 'hum', 'hour']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical),
        ('scaling', MinMaxScaler(), continuous),
        ('poly', PolynomialFeatures(degree=2), continuous)
    ],
    remainder='passthrough'
)

We will also want a way to compare the accuracy of different models across iterations, so we will store them in a dataframe for now

In [35]:
station_numbers = df['number'].unique()
df_results = pd.DataFrame({"number": station_numbers})

## Linear Regression

Let's keep it simple and start with a linear regression

In [36]:
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Add Linear Regression as the final step
])

In [37]:
lr_results = []

for station_number in station_numbers:
    df_station = df[df['number'] == station_number]
    X_train, X_test, y_train, y_test = train_test_split(df_station[categorical + continuous], df_station['availability'], test_size=0.2, random_state=69)
    score = lr_pipeline.fit(X_train, y_train).score(X_test, y_test)
    print("Station Number", station_number, "R2 Score:", score)

    with open(f'models/lr/lr_{station_number}.pkl','wb') as handle:
        pickle.dump(lr_pipeline, handle, pickle.HIGHEST_PROTOCOL)
    
    lr_results.append(score)

df_results['linear_regression'] = lr_results

Station Number 97 R2 Score: 0.5553117195913462
Station Number 53 R2 Score: 0.6099483674709518
Station Number 58 R2 Score: 0.6697957609609182
Station Number 24 R2 Score: 0.4497467980468818
Station Number 44 R2 Score: 0.5910743465148196
Station Number 74 R2 Score: 0.6176444358231167
Station Number 20 R2 Score: 0.7490013735010648
Station Number 80 R2 Score: 0.36049802959972965
Station Number 29 R2 Score: 0.6637826410560037
Station Number 47 R2 Score: 0.6392266595836015
Station Number 88 R2 Score: 0.475702948512135
Station Number 73 R2 Score: 0.5503396142927786
Station Number 3 R2 Score: 0.6051475580131813
Station Number 42 R2 Score: 0.5756270623713888
Station Number 117 R2 Score: 0.5693637014711471
Station Number 71 R2 Score: 0.17213644851783427
Station Number 78 R2 Score: 0.40409345925935714
Station Number 91 R2 Score: 0.6601150186302039
Station Number 22 R2 Score: 0.4265436548523246
Station Number 33 R2 Score: 0.5406624681411538
Station Number 62 R2 Score: 0.5456648233087582
Station Num

## Neural Network

When we included polynomial features, it seemed to be at odds with the non-linearity already introduced by the neural network, so we will use a different preprocessor here and drop the polynomial transformation

In [38]:
nn_preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), categorical),
        ('scaling', MinMaxScaler(), continuous)
    ],
    remainder='passthrough'
)

In [39]:
nn_pipeline = Pipeline([
    ('preprocessor', nn_preprocessor),
    ('regressor', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=69))  # Add Linear Regression as the final step
])

In [40]:
nn_results = []

for station_number in station_numbers:
    df_station = df[df['number'] == station_number]
    X_train, X_test, y_train, y_test = train_test_split(df_station[categorical + continuous], df_station['availability'], test_size=0.2, random_state=69)
    score = nn_pipeline.fit(X_train, y_train).score(X_test, y_test)
    print("Station Number", station_number, "R2 Score:", score)

    with open(f'models/nn/nn_{station_number}.pkl','wb') as handle:
        pickle.dump(nn_pipeline, handle, pickle.HIGHEST_PROTOCOL)

    nn_results.append(score)

df_results['neural_network'] = nn_results

Station Number 97 R2 Score: 0.8443329907079258
Station Number 53 R2 Score: 0.6856288635041903
Station Number 58 R2 Score: 0.9317674841567822
Station Number 24 R2 Score: 0.8173147078537744
Station Number 44 R2 Score: 0.7158242610555311
Station Number 74 R2 Score: 0.7535641635236064
Station Number 20 R2 Score: 0.8893776885159482
Station Number 80 R2 Score: 0.4708927048537904
Station Number 29 R2 Score: 0.783947318815115
Station Number 47 R2 Score: 0.8810171744683251
Station Number 88 R2 Score: 0.6893806252970079
Station Number 73 R2 Score: 0.8261624070124605
Station Number 3 R2 Score: 0.8520421737523076
Station Number 42 R2 Score: 0.8716133980567089
Station Number 117 R2 Score: 0.7769051229967504
Station Number 71 R2 Score: 0.31624247139158124
Station Number 78 R2 Score: 0.4059751795927634
Station Number 91 R2 Score: 0.8212601833762976
Station Number 22 R2 Score: 0.7950687670674305
Station Number 33 R2 Score: 0.7159711640455948
Station Number 62 R2 Score: 0.7446052796514342
Station Numbe

Granted, very little model tweaking was done here to improve the neural network, at a glance the R2 score performance is impressive seems to outperform the linear regression

## Random Forest

The other option for regression is a random forest, it will take the same scaling and polynomial features as the linear regression

In [44]:
rfr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=50))  # Add Linear Regression as the final step
])

In [45]:
rfr_results = []

for station_number in station_numbers:
    df_station = df[df['number'] == station_number]
    X_train, X_test, y_train, y_test = train_test_split(df_station[categorical + continuous], df_station['availability'], test_size=0.2, random_state=69)
    score = rfr_pipeline.fit(X_train, y_train).score(X_test, y_test)
    print("Station Number", station_number, "R2 Score:", score)

    with open(f'models/rfr/rfr_{station_number}.pkl','wb') as handle:
        pickle.dump(nn_pipeline, handle, pickle.HIGHEST_PROTOCOL)

    rfr_results.append(score)

df_results['random_forest_regressor'] = rfr_results

Station Number 97 R2 Score: 0.9595050454037953
Station Number 53 R2 Score: 0.9582670598095488
Station Number 58 R2 Score: 0.9723109565476727
Station Number 24 R2 Score: 0.9190025624317414
Station Number 44 R2 Score: 0.9006396384642045
Station Number 74 R2 Score: 0.9614163836815669
Station Number 20 R2 Score: 0.9506439430759126
Station Number 80 R2 Score: 0.9383671212006025
Station Number 29 R2 Score: 0.9664659836479433
Station Number 47 R2 Score: 0.9825862309963109
Station Number 88 R2 Score: 0.9088927374765468
Station Number 73 R2 Score: 0.9251452305313026
Station Number 3 R2 Score: 0.9437672729664222
Station Number 42 R2 Score: 0.9517655210869992
Station Number 117 R2 Score: 0.9674268433268808
Station Number 71 R2 Score: 0.8813593322257691
Station Number 78 R2 Score: 0.9459111819364856
Station Number 91 R2 Score: 0.9401738398952326
Station Number 22 R2 Score: 0.9186857281415297
Station Number 33 R2 Score: 0.8289097196828566
Station Number 62 R2 Score: 0.9551534063634712
Station Numbe

I was sceptical that we could do much better than the neural network but the random forest achieved exceptional results

In [46]:
"Linear Regression avg. R^2 Score: " + str(df_results['linear_regression'].mean())

'Linear Regression avg. R^2 Score: 0.5595813236509322'

In [47]:
"Neural Network avg. R^2 Score: " + str(df_results['neural_network'].mean())

'Neural Network avg. R^2 Score: 0.7546309222124201'

In [48]:
"Random Forest Regressor avg. R^2 Score: " + str(df_results['random_forest_regressor'].mean())

'Random Forest Regressor avg. R^2 Score: 0.9363755529417217'